<a href="https://colab.research.google.com/github/qubit55/CL-CXX-JIT/blob/main/embeddings_clojure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First install Clojupyter kernel

In [1]:
!wget https://raw.githubusercontent.com/qubit55/clojupyter_colab_setup/refs/heads/main/install_clojure_kernel.sh
!chmod +x install_clojure_kernel.sh
!./install_clojure_kernel.sh

--2025-03-01 23:07:27--  https://raw.githubusercontent.com/qubit55/clojupyter_colab_setup/refs/heads/main/install_clojure_kernel.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1698 (1.7K) [text/plain]
Saving to: ‘install_clojure_kernel.sh’

install_clojure_ker 100%[===================>]   1.66K  --.-KB/s    in 0s      

2025-03-01 23:07:27 (31.1 MB/s) - ‘install_clojure_kernel.sh’ saved [1698/1698]

JAVA_HOME is set to: /root/.jdk/jdk-21.0.6+7
openjdk version "21.0.6" 2025-01-21 LTS
OpenJDK Runtime Environment Temurin-21.0.6+7 (build 21.0.6+7-LTS)
OpenJDK 64-Bit Server VM Temurin-21.0.6+7 (build 21.0.6+7-LTS, mixed mode, sharing)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload

# Once the installation of Clojupyter is complete

* Go to Runtime -> Change runtime type.
* In the "Runtime type" dropdown, select Clojure IPC.
* Wait for the kernel to initialize. The RAM and disk usage chart in the upper-right corner will confirm when it’s ready.
* Now you can start executing the cells down below

In [1]:
(require '[clojupyter.misc.helper :as helper]
         '[clojupyter.display :as display])

(helper/add-dependencies '[com.knuddels/jtokkit "1.1.0"])
(helper/add-dependencies '[criterium "0.4.6"])
(helper/add-dependencies '[uncomplicate/deep-diamond "0.31.0"])
(helper/add-dependencies '[uncomplicate/neanderthal "0.53.2"])
(helper/add-dependencies '[uncomplicate/fluokitten "0.10.0"])

uncomplicate/fluokitten 0.10.0 org.clojure/clojure 1.11.3 org.clojure/clojure 1.11.3 org.clojure/core.specs.alpha 0.2.62 org.clojure/spec.alpha 0.3.218 org.clojure/spec.alpha 0.3.218 org.clojure/core.specs.alpha 0.2.62

In [3]:
(import '[com.knuddels.jtokkit Encodings]
        '[com.knuddels.jtokkit.api EncodingType IntArrayList])

(require '[clojure.repl :refer [doc]]
         '[criterium.core :refer [quick-bench]]
         '[uncomplicate.fluokitten.core :refer [fmap!]]
         '[uncomplicate.commons.core :refer [with-release]]
         '[uncomplicate.neanderthal
           [native :refer [dv dge fge dv fv iv]]
           [core :refer [mv! dim entry row rows col cols submatrix subvector ncols mrows transfer! transfer]]
           [random :refer [rand-uniform!]]])

nil

In [6]:
(def emb-mat (fge 50000 1000))
(rand-uniform! emb-mat)

#RealGEMatrix[float, mxn:50000x1000, layout:column]

In [7]:
(quick-bench (rand-uniform! emb-mat))

Evaluation count : 24 in 6 samples of 4 calls.
             Execution time mean : 28.044564 ms
    Execution time std-deviation : 802.331046 µs
   Execution time lower quantile : 27.473189 ms ( 2.5%)
   Execution time upper quantile : 29.398645 ms (97.5%)
                   Overhead used : 10.206787 ns

Found 1 outliers in 6 samples (16.6667 %)
	low-severe	 1 (16.6667 %)
 Variance from outliers : 13.8889 % Variance is moderately inflated by outliers


nil

In [8]:
(quick-bench (row emb-mat 100))

Evaluation count : 1197708 in 6 samples of 199618 calls.
             Execution time mean : 592.388508 ns
    Execution time std-deviation : 183.734154 ns
   Execution time lower quantile : 461.061037 ns ( 2.5%)
   Execution time upper quantile : 810.194359 ns (97.5%)
                   Overhead used : 10.206787 ns


nil

In [9]:
(defn pluck-embeddings
 [tok-ids emb-mat]
 (let
  [emb-mat-subset (fge (count tok-ids) (ncols emb-mat))]
  (loop [i 0 tok-id tok-ids]
   (if (< i (mrows emb-mat-subset))
    (do
      (transfer! (row emb-mat (first tok-id)) (row emb-mat-subset i))
      (recur (inc i) (rest tok-id)))))
   emb-mat-subset))

#'user/pluck-embeddings

In [ ]:
(quick-bench (pluck-embeddings [20 100] emb-mat))

Evaluation count : 84 in 6 samples of 14 calls.
             Execution time mean : 21.472114 ms
    Execution time std-deviation : 5.085178 ms
   Execution time lower quantile : 15.868418 ms ( 2.5%)
   Execution time upper quantile : 26.695136 ms (97.5%)
                   Overhead used : 12.070230 ns


nil

In [ ]:
(quick-bench (row emb-mat 100))

Evaluation count : 1337718 in 6 samples of 222953 calls.
             Execution time mean : 721.712860 ns
    Execution time std-deviation : 198.659871 ns
   Execution time lower quantile : 440.552726 ns ( 2.5%)
   Execution time upper quantile : 939.877270 ns (97.5%)
                   Overhead used : 12.070230 ns


nil